In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datetime
import pickle
import os.path
import base64
import pandas as pd
from io import BytesIO
import os
import openpyxl
import mysql.connector
from sshtunnel import SSHTunnelForwarder
import pymysql
import mysql.connector
from requests.structures import CaseInsensitiveDict
import requests

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


#Function to handle authentication and authorization to access the Gmail API using OAuth 2.0
def get_gmail_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/client_secret_811893502700-tj4c2bv2942q1ua3459au95sc1ohoqle.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

#Function to search for a message using gmail address, attachment contained in the email etc. 
def search_messages(service, query):
    result = service.users().messages().list(userId='me', q=query).execute()
    messages = []
    if 'messages' in result:
        messages.extend(result['messages'])
    return messages


#Function to download a specific attachment from a Gmail message and saves it to a specified directory
def get_attachments(service, message_id, store_dir, desired_filename):
    try:
        message = service.users().messages().get(userId='me', id=message_id).execute()
        parts = message.get('payload', {}).get('parts', [])
        for part in parts:
            if part['filename'] == desired_filename:  # Check if the filename matches the desired one
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body'].get('attachmentId')
                    att = service.users().messages().attachments().get(userId='me', messageId=message_id, id=att_id).execute()
                    data = att['data']

                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                path = os.path.join(store_dir, part['filename'])

                if not os.path.exists(store_dir):
                    os.makedirs(store_dir)
                with open(path, 'wb') as f:
                    f.write(file_data)
    except Exception as e:
        print(f"An error occurred: {e}")


current_date = datetime.date.today()
current_week = current_date.isocalendar()[1]
previous_week = current_week - 1
current_year = datetime.date.today().year

new_file_path = ''


def main():
    service = get_gmail_service()
    search_query = 'from:veysel_kaya@hotmail.com has:attachment filename:SUNSEEDMANGEMIVAGON20232024.xlsx'
    messages = search_messages(service, search_query)

    if messages:
        latest_message = messages[0]  # Get the latest email

        # Construct the path for saving the attachment with the week and year in the file name
        #folder_name = f'{current_year}_Week_{previous_week}'
        save_path = '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/TRK_imports'
        

        # Call get_attachments with the desired filename
        get_attachments(service, latest_message['id'], save_path, 'SUNSEEDMANGEMIVAGON20232024.xlsx')

        # Path to the downloaded Excel file
        old_file_path = os.path.join(save_path, 'SUNSEEDMANGEMIVAGON20232024.xlsx')
        new_file_path = os.path.join(save_path, f'{current_year}_Week_{previous_week}.xlsx')

        try:
            # Rename the downloaded file
            os.rename(old_file_path, new_file_path)
            return new_file_path

            # excel_data = pd.read_excel(new_file_path)
            # print(excel_data)
        except Exception as e:
            print(f"Error reading or renaming the Excel file: {e}")

if __name__ == '__main__':
    new_file_path = main()
    print(f"New file path: {new_file_path}")


New file path: /Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/TRK_imports/2024_Week_8.xlsx


In [2]:

# The 'dates' list contains mappings of date labels as they appear in a crawled Excel file to their corresponding ISO 8601 formatted date strings. The date labels in the keys are in a mixed 
# format with Turkish and English month names followed by the year, while the values are the last day of the corresponding month in the YYYY-MM-DD format. This conversion is useful for:
# 1. Clarity: The ISO 8601 date format (YYYY-MM-DD) is more standardized and easier to work with in data processing and database operations.
# 2. Data Retrieval: Allows for easy and accurate retrieval of lines of data associated with each date from the Excel file.
# 3. Database Insertion: Simplifies the process of inserting dates into databases, as the ISO format is widely accepted and used in most database systems.

dates = [
    {"Temmuz/July 2023": "2023-07-31"},
    {"Ağustos/August 2023": "2023-08-31"},
    {"Eylül/September 2023": "2023-09-30"},
    {"Ekim/October 2023": "2023-10-31"},
    {"Kasım/November 2023": "2023-11-30"},
    {"Aralık/December 2023": "2023-12-31"},
    {"Ocak/January 2024": "2024-01-31"},
    {"Şubat/February 2024": "2024-02-28"},
    {"Mart/March 2024": "2024-03-31"},
    {"Nisan/April 2024": "2024-04-30"},
    {"Mayıs/May 2024": "2024-05-31"},
    {"Haziran/June 2024": "2024-06-30"}
]

# The 'countries' list contains mappings of country names as they appear in the crawled Excel file to their corresponding country codes used in the database. 
# The country names in the keys are in a mixed format with Turkish and English names followed by their ISO 3166-1 alpha-2 codes, while the values are the corresponding country codes 
# from the database. This conversion is necessary for:
# 1. Consistency: Ensures that the country names from the Excel file match the standardized country codes in the database, maintaining consistency in the data representation.
# 2. Data Retrieval: Allows for accurate retrieval of lines of data associated with each country from the Excel file.
# 3. Database Insertion: Simplifies the process of inserting data into the database, as the country codes are standardized 
# 4. Despite the method being bulky, it is necessary to handle the varied country names and codes properly for seamless data integration between the Excel file and the database

countries = [
    {"Ukrayna/UA": 184},
    {"Rusya/RU": 150},
    {"Moldova/MD": 119},
    {"Romanya/RO": 149},
    {"Avusturya/AT": 14},
    {"Azerbaycan/AZ": 15},
    {"Bahamas/BS": 16},
    {"Bahreyn/BH": 17},
    {"Bangladeş/BD": 18},
    {"Barbados/BB": 19},
    {"Belarus/BY": 20},
    {"Belçika/BE": 21},
    {"Belize/BZ": 22},
    {"Benin/BJ": 23},
    {"Bermuda/BM": 24},
    {"Butan/BT": 25},
    {"Bolivya/BO": 26},
    {"Bosna Hersek/BOS": 27},
    {"Botsvana/BW": 28},
    {"Brezilya/BR": 29},
    {"Bulgaristan/BG": 30},
    {"Burkina Faso/BF": 31},
    {"Burundi/BI": 32},
    {"Kamboçya/KH": 33},
    {"Kamerun/CM": 34},
    {"Cayman Adaları/KY": 35},
    {"Orta Afrika Cumhuriyeti/CF": 36},
    {"Çad/TD": 37},
    {"Şili/CL": 38},
    {"Çin/CN": 39},
    {"Kolombiya/CO": 40},
    {"Komorlar/KM": 41},
    {"Kongo DC/CD": 42},
    {"Kosta Rika/CR": 43},
    {"Hırvatistan/HR": 44},
    {"Küba/CU": 45},
    {"Kıbrıs/CY": 46},
    {"Çek Cumhuriyeti/CZ": 47},
    {"Danimarka/DK": 48},
    {"Cibuti/DJ": 49},
    {"Dominika/DM": 50},
    {"Dominik Cumhuriyeti/DO": 51},
    {"Ekvador/EC": 52},
    {"Mısır/EG": 53},
    {"El Salvador/SV": 54},
    {"Ekvator Ginesi/GQ": 55},
    {"Eritre/ER": 56},
    {"Estonya/EE": 57},
    {"Etiyopya/ET": 58},
    {"Faroe Adaları/FO": 59},
    {"Fiji/FJ": 60},
    {"Finlandiya/FI": 61},
    {"Fransa/FR": 62},
    {"Gabon/GA": 63},
    {"Gambiya/GM": 64},
    {"Gürcistan/GEOR": 65},
    {"Almanya/DE": 66},
    {"Gana/GH": 67},
    {"Cebelitarık/GI": 68},
    {"Yunanistan/GR": 69},
    {"Grönland/GL": 70},
    {"Grenada/GD": 71},
    {"Guam/GU": 72},
    {"Guatemala/GT": 73},
    {"Guyana/GY": 74},
    {"Haiti/HT": 75},
    {"Honduras/HN": 76},
    {"Hong Kong SAR/HK": 77},
    {"Macaristan/HU": 78},
    {"İzlanda/IS": 79},
    {"Hindistan/IN": 80},
    {"Endonezya/ID": 81},
    {"İran/IR": 82},
    {"Irak/IQ": 83},
    {"İrlanda/IE": 84},
    {"İsrail/IL": 85},
    {"İtalya/IT": 86},
    {"Jamaika/JM": 87},
    {"Japonya/JP": 88},
    {"Ürdün/JO": 89},
    {"Kazakistan/KZ": 90},
    {"Kenya/KE": 91},
    {"Kiribati/KI": 92},
    {"Güney Kore/KR": 93},
    {"Kosova/XK": 94},
    {"Kuveyt/KW": 95},
    {"Kırgızistan/KG": 96},
    {"Laos/LA": 97},
    {"Letonya/LV": 98},
    {"Lübnan/LB": 99},
    {"Lesotho/LS": 100},
    {"Liberya/LR": 101},
    {"Libya/LY": 102},
    {"Lihtenştayn/LI": 103},
    {"Litvanya/LT": 104},
    {"Lüksemburg/LU": 105},
    {"Macao SAR/MO": 106},
    {"Madagaskar/MG": 107},
    {"Malavi/MW": 108},
    {"Malezya/MY": 109},
    {"Maldivler/MV": 110},
    {"Mali/ML": 111},
    {"Malta/MT": 112},
    {"Marshall Adaları/MH": 113},
    {"Moritanya/MR": 114},
    {"Mauritius/MU": 115},
    {"Meksika/MX": 116},
    {"Moldova/MD": 117},
    {"Monako/MC": 118},
    {"Mongolistan/MN": 119},
    {"Karadağ/ME": 120},
    {"Fas/MA": 121},
    {"Mozambik/MZ": 122},
    {"Myanmar/MM": 123},
    {"Namibya/NA": 124},
    {"Nauru/NR": 125},
    {"Nepal/NP": 126},
    {"Hollanda/NL": 127},
    {"Yeni Kaledonya/NC": 128},
    {"Yeni Zelanda/NZ": 129},
    {"Nikaragua/NI": 130},
    {"Nijer/NE": 131},
    {"Nijerya/NG": 132},
    {"Norveç/NO": 133},
    {"Umman/OM": 134},
    {"Pakistan/PK": 135},
    {"Palau/PW": 136},
    {"Panama/PA": 137},
    {"Papua Yeni Gine/PG": 138},
    {"Paraguay/PY": 139},
    {"Peru/PE": 140},
    {"Filipinler/PH": 141},
    {"Polonya/PL": 142},
    {"Portekiz/PT": 143},
    {"Porto Riko/PR": 144},
    {"Katar/QA": 145},
    {"Romanya/RO": 146},
    {"Ruanda/RW": 148},
    {"Saint Kitts ve Nevis/KN": 149},
    {"Saint Lucia/LC": 150},
    {"Saint Vincent ve Grenadinler/VC": 151},
    {"Saint Pierre ve Miquelon/PM": 152},
    {"Samoa/WS": 153},
    {"San Marino/SM": 154},
    {"Sao Tome ve Principe/ST": 155},
    {"Suudi Arabistan/SA": 156},
    {"Senegal/SN": 157},
    {"Sırbistan/SER": 158},
    {"Seyşeller/SC": 159},
    {"Sierra Leone/SL": 160},
    {"Singapur/SG": 161},
    {"Slovakya/SK": 162},
    {"Slovenya/SI": 163},
    {"Solomon Adaları/SB": 164},
    {"Somali/SO": 165},
    {"Güney Afrika/ZA": 166},
    {"İspanya/ES": 167},
    {"Sri Lanka/LK": 168},
    {"Sudan/SD": 169},
    {"Surinam/SR": 170},
    {"Svaziland/SZ": 171},
    {"İsveç/SE": 172},
    {"İsviçre/CH": 173},
    {"Suriye/SY": 174},
    {"Tayvan/TW": 175},
    {"Tacikistan/TJ": 176},
    {"Tanzanya/TZ": 177},
    {"Tayland/TH": 178},
    {"Togo/TG": 179},
    {"Tonga/TO": 180},
    {"Trinidad ve Tobago/TT": 181},
    {"Tunus/TN": 182},
    {"Türkiye/TR": 183},
    {"Türkmenistan/TM": 184},
    {"Tuvalu/TV": 185},
    {"Uganda/UG": 186},
    {"Birleşik Arap Emirlikleri/AE": 188},
    {"Uruguay/UY": 189},
    {"Özbekistan/UZ": 190},
    {"Vanuatu/VU": 191},
    {"Venezuela/VE": 192},
    {"Vietnam/VN": 193},
    {"Virgin Adaları, Britanya/VG": 194},
    {"Virgin Adaları, Amerika/VI": 195},
    {"Yemen/YE": 196},
    {"Zambiya/ZM": 197},
    {"Zimbabve/ZW": 198},
    {"Avrupa 28/EU-28": 199},
    {"Avrupa 27/EU-27": 200},
    {"Birleşik Krallık/UK": 201},
    {"Küresel Süt Ticareti/GDT": 202},
    {"Fransa - ATLA/ATLA": 203},
    {"Beneluks/BE-NE-LU": 204},
    {"Küresel/Global": 205},
    {"Doğu AB/East-EU": 206},
    {"LATAM/LATAM": 207},
    {"Okyanusya/Oceania": 208},
    {"ABD CLS raporu/US CLS report": 209},
    {"ABD - NDP/United States - NDP": 210},
    {"Cape Verde/CV": 211},
    {"Brunei/BN": 212},
    {"Cook Adaları/CK": 213},
    {"Fildişi Sahili/CI": 214},
    {"Fransız Polinezyası/PF": 215},
    {"Gine/GN": 216},
    {"Gine-Bissau/GW": 217},
    {"Kuzey Kore/KP": 218},
    {"Mayotte/YT": 219},
    {"Mikronezya/FS": 220},
    {"Saint Kitts ve Nevis/KN": 221},
    {"Saint Lucia/LC": 222},
    {"Saint Vincent ve Grenadinler/VC": 223},
    {"Saint Pierre ve Miquelon/PM": 224},
    {"Samoa/WS": 225},
    {"Swaziland/SZ": 226},
    {"Doğu Timor/TL": 227},
    {"Wallis ve Futuna Adaları/WF": 228},
    {"Belirsiz/Unspecified": 229},
    {"Avrupa Ekonomik Alanı/EEA": 230},
    {"Britanya Hint Okyanusu Toprakları/IO": 231},
    {"Fransız Güney Toprakları/TF": 232},
    {"Asya/Asia": 233},
    {"Mercosur/Mercosur": 234},
    {"Orta Doğu/Middle East": 235},
    {"Kuzey Afrika/North Africa": 236},
    {"Kuzey Amerika/North America": 237},
    {"Güney Amerika/South America": 238},
    {"Orta Amerika/Central America": 239},
    {"Kuzeydoğu Asya/Northeast Asia": 240},
    {"Merkez Güney Asya/Central South Asia": 241},
    {"Güneydoğu Asya/Southeast Asia": 242},
    {"EAEB/EAEU": 243},
    {"Sahra Altı Afrika/Sub-saharan Africa": 244},
    {"Dünya/World": 245},
    {"ABD (Ortabatı)/United States (Midwest)": 246},
    {"ABD (Kuzeydoğu)/United States (Northeast)": 247},
    {"ABD (Güney)/United States (South)": 248},
    {"ABD (Batı)/United States (West)": 249},
    {"Batı AB/West-EU": 250}
]



#Connect to the database into which to insert the data from the crawled excel file using SSHTunnelForwarder
b_ssh_host = '18.195.40.197'
b_ssh_user = 'forge'
b_ssh_port = 22
b_ssh_private_key = '/Users/germankosenkov/.ssh/id_rsa'
b_sql_hostname = 'gehrman.ccq0tjftm3pw.eu-central-1.rds.amazonaws.com'
b_sql_username = 'gehrman'
b_sql_password = 'ycEqAKjqFZH3sH'
b_sql_database = 'vesper'
b_sql_port = 3306

with SSHTunnelForwarder(
        (b_ssh_host, b_ssh_port),
        ssh_username=b_ssh_user,
        ssh_pkey=b_ssh_private_key,
        remote_bind_address=(b_sql_hostname, b_sql_port)) as tunnel:

    b_conn = mysql.connector.connect(
        host='127.0.0.1',
        user=b_sql_username,
        passwd=b_sql_password,
        db=b_sql_database,
        port=tunnel.local_bind_port
    )

    b_cursor = b_conn.cursor()

    # Path to the Excel file
    workbook = openpyxl.load_workbook(new_file_path, data_only=True)
    # Select the sheet to extract data from
    sheet = workbook['TOPLAM']  # You can also specify the sheet name like this: sheet = workbook['Sheet1']

    # Iterate through the rows and extract data from columns month,quantity,country_of_origin
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # Assuming columns A, B, and C correspond to index 0, 1, and 2
        month = row[5]#changed
        quantity = row[6]
        country_of_origin = row[7]

        #column_b_value = row[5]#changed
        #column_c_value = row[6]
        #column_d_value = row[7]

        # Check if the values are not headers and not None
        if month and quantity and country_of_origin:
            if month != "Ay" and quantity != "Miktar" and country_of_origin != "Ülke":
                # Find the corresponding date and country values from dictionaries
                for date_dict in dates:
                    for key, value in date_dict.items():
                        if key == month:
                            month = value
                            break

                for country_dict in countries:
                    for key, value in country_dict.items():
                        if key == country_of_origin:
                            country_of_origin = value
                            break

                #Before inserting check if the data already exists in the table
                #If exists we will update the row as in the new coming files data can be revised by the source
                #if data doesn't exist we insert a new row 
                check_data_query = "SELECT * FROM TradeData WHERE product_id = %s AND date = %s AND raw_trade_quantity = %s AND origin_country_id = %s AND destination_country_id_raw = %s AND data_source_id = %s"
                val = (859, month, quantity, country_of_origin, 180, 4)
                b_cursor.execute(check_data_query, val)
                existing_data = b_cursor.fetchone() 

                try:
                    b_conn.autocommit = False

                    if existing_data:
                        print("Replacing existing data for", month)
                        update_query = "UPDATE TradeData SET raw_trade_quantity = %s, destination_country_id_raw = %s, origin_country_id = %s, trade_quantity = %s, raw_unit = %s, unit = %s, frequency = %s WHERE product_id = %s AND date = %s AND data_source_id = %s AND destination_country_id_raw = %s AND origin_country_id = %s"
                        update_values = (quantity, 180, country_of_origin, quantity, 'mt', 'mt', 'monthly', 859, month, 4, 180, country_of_origin)
                        b_cursor.execute(update_query, update_values)
                    else:
                        print("Inserting data for", month)
                        insert_query = "INSERT INTO TradeData (product_id, date, raw_trade_quantity, destination_country_id_raw, origin_country_id, trade_quantity, raw_unit, unit, data_source_id, frequency) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        insert_values = (859, month, quantity, 180, country_of_origin, quantity, 'mt', 'mt', 4, 'monthly')
                        b_cursor.execute(insert_query, insert_values)

                    b_conn.commit()  # Commit the transaction
                
                except Exception as e:
                    b_conn.rollback()  # Rollback the transaction if an error occurs
                    print("Error occurred:", e)

                finally:
                    b_conn.autocommit = True


    b_cursor.close()
    b_conn.close()

print(b_cursor.rowcount, "record(s) inserted.")



Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-10-31
Replacing existing data for 2023-10-31
Replacing existing data for 2023-10-31
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data f